<a href="https://colab.research.google.com/github/karthikdoguparthi/Amazon-Prime-EDA/blob/main/Amazon_Prime_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - Amazon Prime EDA



##### **Project Type**    - EDA
##### **Contribution**    - Individual
##### **Team Member 1 -** Karthik Doguparthi


# **Project Summary -**

Write the summary here within 500-600 words.

# **GitHub Link -**

https://github.com/karthikdoguparthi/Amazon-Prime-EDA

# **Problem Statement**


**Write Problem Statement Here.**

#### **Define Your Business Objective?**

Answer Here.

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required.
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits.
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 20 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule.

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import zipfile
import pandas as pd
import numpy as np
import os
import matplotlib
import ast
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter

### Dataset Loading

In [ ]:
#Mounting drive
from google.colab import drive
drive.mount('/content/drive')

# Load Dataset
with zipfile.ZipFile('/content/drive/MyDrive/Labmentix/AmazonPrimeEDA/credits.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/Labmentix/AmazonPrimeEDA/')

with zipfile.ZipFile('/content/drive/MyDrive/Labmentix/AmazonPrimeEDA/titles.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/Labmentix/AmazonPrimeEDA/')


df1 = pd.read_csv('/content/drive/MyDrive/Labmentix/AmazonPrimeEDA/credits.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Labmentix/AmazonPrimeEDA/titles.csv')


#merging two datsets
merged_df = pd.merge(df1, df2, on='id', how='inner')

### Dataset First View

In [ ]:
# Dataset First Look
merged_df.head()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
rows, cols = merged_df.shape
print(f"Number of rows: {rows}")
print(f"Number of columns: {cols}")

### Dataset Information

In [ ]:
# Dataset Info
merged_df.info()

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count

merged_df.duplicated().sum()

In [ ]:
merged_df[merged_df.duplicated()]

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count wrt dataset

merged_df.isnull().sum()

In [ ]:
# Missing Values/Null Values Total Count

merged_df.isnull().sum().sum()

In [ ]:
# Visualizing the missing values

merged_df[merged_df.isnull().any(axis=1)]

### What did you know about your dataset?

After loading and merging my dataset, I noticed this was a large datset with 124347 rows and 19 columns within. When I saw the datset information I could find that count of all the columns are not same. Which imples there are NULL/NaN values in the dataset. Well what else can we hope from a real time dataset.

I also noticed duplicate values in the dataset ( 168 duplicates values in the whole datset) mostly which were referring to the same 'id' and the only change was the actor from the show. NaN values are from Age Certification, Seasons of the show and IMDB ratings.

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns

merged_df.columns

In [ ]:
# Dataset Describe

merged_df.describe(include='all')

### Variables Description

The dataset consists of list of shows avalilable in Amazon Prime, with each row identifying by 'id'. It contains variables like name, character, title of the show, release year, imdb rating on the scale of 10. Age certification shows the minimum age limit to watch the movie, if below 13 parental guidance(TV-PG) or below 18 or R rated etc. The genre column shows the set of all the genres included by the show. These are about few variables given in the dataset.

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values Count for each variable.

merged_df.nunique()

In [ ]:
# Check Unique Values for each variable.

for column in merged_df.columns:
    unique_values = merged_df[column].unique()
    print(f"Unique values for column '{column}':")
    print(unique_values)
    print()

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Write your code to make your dataset analysis ready.

# Delete Description column
merged_df.drop('description', axis=1, inplace=True)

# Delete IMDB ID column
merged_df.drop('imdb_id', axis=1, inplace=True)

# Delete Character column
merged_df.drop('character', axis=1, inplace=True)

# Fill NaN with 0
merged_df['seasons'] = merged_df['seasons'].fillna(0)

#Fill remaining NaN in  dataset with 0
merged_df = merged_df.fillna(0)

# Fill NaNs with 'TV-PG'
merged_df['age_certification'] = merged_df['age_certification'].fillna('TV-PG')

# Convert values > 21 to 21, then restore 'TV-PG'
merged_df['age_certification'] = (
    pd.to_numeric(merged_df['age_certification'], errors='coerce')
    .clip(upper=21)
    .fillna('TV-PG')
)

# Step 1: Parse strings to lists or dictionaries safely
def extract_countries(val):
    try:
        parsed = ast.literal_eval(val) if isinstance(val, str) else val
        if isinstance(parsed, dict) and not parsed:
            return None  # Empty dict
        elif isinstance(parsed, list) and not parsed:
            return None  # Empty list
        elif isinstance(parsed, list):
            return parsed
        elif isinstance(parsed, str):
            return [parsed]
        else:
            return None
    except:
        return None

# Apply parser
merged_df['parsed_countries'] = merged_df['production_countries'].apply(extract_countries)

# Drop null or empty results
cleaned_df = merged_df[merged_df['parsed_countries'].notnull()]

# Explode into individual country rows
exploded = cleaned_df.explode('parsed_countries')

# Count top 10 countries
top_countries = exploded['parsed_countries'].value_counts().head(10)

In [ ]:
# Verifying we have all Non Null values
merged_df.info()

### What all manipulations have you done and insights you found?

I have gone through all the columns in the dataset and removed the columns with which we cannot have any insights i.e Description, IMDB_ID, Character. Filled NaN in Seasons with 0 because as the type suggested Show/Movie, Movie will have only 1 season and few other shows might also have only 1 season. It is better to replace that with 0. With this only few NaN will be left in imdb_scores, imdb_votes, tmdb_popularity, tmdb_score. I replaced all those NaN with 0 to get better results when any data visualisation is performed on the colomn.

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Chart - 1

In [ ]:
# Chart - 1 visualization code
# What type of show it is
plt.figure(figsize=(5,5))
merged_df['type'].value_counts().plot(kind='pie',autopct='%1.1f%%',
    colors=['lightblue', 'seagreen'], wedgeprops={'edgecolor': 'black'})

plt.title('Distribution of Content Type')
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 2

In [ ]:
# Chart - 2 visualization code
# Top 10 most available shows wrt genre

# Convert genre strings to real Python lists, handling non-string values
merged_df['genres'] = merged_df['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Set genre dictionary so each row has only one genre
exploded_df = merged_df.explode('genres')

# Plot graph
exploded_df['genres'].value_counts().head(10).plot(kind='bar', title='Top 10 Genres by Count', xlabel='Genre', ylabel='Frequency', figsize=(10,5))
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 3

In [ ]:
# Chart - 3 visualization code

# Release year trend how content releases have changed over time

plt.figure(figsize=(12,6))
merged_df['release_year'].value_counts().sort_index().plot(kind='line',color='teal')

plt.title('Number of Releases by Year')
plt.xlabel('Release Year')
plt.ylabel('Count')
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 4

In [ ]:
# Chart - 4 visualization code

# Drop NaNs and ensure everything is a list
genre_series = merged_df['genres'].dropna().apply(lambda x: x if isinstance(x, list) else str(x).strip("[]").replace("'", "").split(','))
all_genres = [genre.strip() for sublist in genre_series for genre in sublist if isinstance(genre, str)]
genre_text = ' '.join(all_genres)

wordcloud = WordCloud(width=800, height=400, background_color='grey').generate(genre_text)
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Top Genres Word Cloud')
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 5

In [ ]:
# Chart - 5 visualization code

# Top actors/roles Most frequent actors or contributors

plt.figure(figsize=(10,6))
merged_df['name'].value_counts().head(10).sort_values().plot(kind='barh',color='seagreen',edgecolor='black')

plt.title('Top 10 Most Frequent Names')
plt.xlabel('Count')
plt.ylabel('Name')
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 6

In [ ]:
# Chart - 6 visualization code

#Role distribution Director vs Writer vs Actor etc
plt.figure(figsize=(10,6))
merged_df['role'].value_counts().head(10).sort_values().plot(kind='barh',color='salmon',edgecolor='black')

plt.title('Role of person')
plt.xlabel('Count')
plt.ylabel('Role')
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 7

In [ ]:
# Chart - 7 visualization code
# IMDB and TMDB score distribution

# Create figure with 1 row and 2 columns
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot imdb_score histogram
axes[0].hist(merged_df['imdb_score'].dropna(), bins=20, alpha=0.7, color='skyblue')
axes[0].set_title('IMDB Score Distribution')
axes[0].set_xlabel('IMDB Score')
axes[0].set_ylabel('Frequency')

# Plot tmdb_score histogram
axes[1].hist(merged_df['tmdb_score'].dropna(), bins=20, alpha=0.7, color='salmon')
axes[1].set_title('TMDB Score Distribution')
axes[1].set_xlabel('TMDB Score')
axes[1].set_ylabel('Frequency')
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 8

In [ ]:
# Chart - 8 visualization code
# Content by Release Year and Type
year_counts = (
    merged_df[merged_df['release_year'].notnull()]
    .groupby(['release_year', 'type'])['title'].count().reset_index()
    .rename(columns={'title': 'count'}))

plt.figure(figsize=(12, 6))
sns.lineplot(data=year_counts, x='release_year', y='count', hue='type')
plt.title('Content Count by Year and Type')
plt.xlabel('Release Year')
plt.ylabel('Number of Titles')
plt.tight_layout()
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 9

In [ ]:
# Chart - 9 visualization code

plt.figure(figsize=(10, 6))
top_countries.sort_values().plot(kind='barh',color='brown',edgecolor='black')
plt.title('Top 10 Production Countries')
plt.xlabel('Number of Titles')
plt.ylabel('Country')
plt.tight_layout()
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 10

In [ ]:
# Chart - 10 visualization code

# Get season number
merged_df['seasons'] = pd.to_numeric(merged_df['seasons'], errors='coerce')
seasons_counts = merged_df['seasons'].value_counts().sort_index()

# Filter to only seasons from 0 to 3
seasons_counts = seasons_counts[(seasons_counts.index >= 0) & (seasons_counts.index <= 3)]
ax = seasons_counts.plot(kind='bar', color='coral', figsize=(8, 5))

# Set labels and title
plt.xlabel('Number of Seasons')
plt.ylabel('Count')
plt.title('Seasons')
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 11

In [ ]:
# Chart - 11 visualization code
# IMDb Score Distribution by Type and Genre shows variability and outliers.
top_genres = exploded_df['genres'].value_counts().head(5).index
subset = exploded_df[exploded_df['genres'].isin(top_genres)]

plt.figure(figsize=(12, 6))
sns.boxplot(data=subset, x='genres', y='imdb_score', hue='type')
plt.title('IMDb Score Distribution for Top Genres by Type')
plt.xlabel('Genre')
plt.ylabel('IMDb Score')
plt.tight_layout()
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 12

In [ ]:
# Chart - 12 visualization code
# Compare genre performance between type
pivot_df = (
    exploded_df[exploded_df['imdb_score'].notnull()]
    .groupby(['genres', 'type'])['imdb_score'].mean().reset_index())

plt.figure(figsize=(12, 6))
sns.barplot(data=pivot_df, x='genres', y='imdb_score', hue='type', palette='Set2')
plt.title('Average IMDb Score by Genre and Type')
plt.xlabel('Genre')
plt.ylabel('Avg IMDb Score')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 13

In [ ]:
# Chart - 13 visualization code

plt.figure(figsize=(10, 6))
sns.scatterplot(data=merged_df, x='runtime', y='imdb_score', hue='type', alpha=0.7)
plt.title('Duration vs IMDb Score by Type')
plt.xlabel('Duration (minutes)')
plt.ylabel('IMDb Score')
plt.tight_layout()
plt.xlim(0, 250)
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 14 - Correlation Heatmap

In [ ]:
# Correlation Heatmap visualization code

exploded_df['decade'] = (exploded_df['release_year'] // 10) * 10

heatmap_data = (
    exploded_df[exploded_df['imdb_score'].notnull()].groupby(['genres', 'decade'])['imdb_score'].mean()
    .unstack().fillna(0))

plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, annot=True, fmt=".1f", cmap='coolwarm')
plt.title('Correlation Heatmap of Avg IMDb Score by Genre and Decade')
plt.xlabel('Decade')
plt.ylabel('Genre')
plt.tight_layout()
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

#### Chart - 15 - Pair Plot

In [ ]:
# Pair Plot visualization code


# Filter to remove outliers and missing values
filtered_df = merged_df[
    (merged_df['runtime'] <= 150) &
    (merged_df['release_year'].notnull()) &
    (merged_df['type'].notnull())
]

# Select relevant columns
pair_data = filtered_df[['runtime', 'release_year', 'type']]

# Create the pair plot with hue
g = sns.pairplot(pair_data, hue='type', kind='scatter', diag_kind='hist', plot_kws={'alpha': 0.6})

# Move the legend to the top-right outside the plot
g._legend.set_bbox_to_anchor((1.05, 1))
g._legend.set_frame_on(True)
g._legend.set_title('Type')

# Add title
plt.suptitle('Pair Plot: Runtime vs Release Year by Type', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

## **5. Solution to Business Objective**

#### What do you suggest the client to achieve Business Objective ?
Explain Briefly.

Answer Here.

# **Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your EDA Capstone Project !!!***